In [1]:
import numpy as np
import pandas as pd

In [2]:
players_stats = pd.read_csv(r"C:\Users\himan\Downloads\DA\Project\NBA_Analytics\Data\Raw\PlayerStatistics.csv")

C:\Users\himan\AppData\Local\Temp\ipykernel_11440\3578262026.py:1: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  players_stats = pd.read_csv(r"C:\Users\himan\Downloads\DA\Project\NBA_Analytics\Data\Raw\PlayerStatistics.csv")


In [3]:
players_stats.shape

(1644443, 35)

In [4]:
players_stats.columns

Index(['firstName', 'lastName', 'personId', 'gameId', 'gameDateTimeEst',
       'playerteamCity', 'playerteamName', 'opponentteamCity',
       'opponentteamName', 'gameType', 'gameLabel', 'gameSubLabel',
       'seriesGameNumber', 'win', 'home', 'numMinutes', 'points', 'assists',
       'blocks', 'steals', 'fieldGoalsAttempted', 'fieldGoalsMade',
       'fieldGoalsPercentage', 'threePointersAttempted', 'threePointersMade',
       'threePointersPercentage', 'freeThrowsAttempted', 'freeThrowsMade',
       'freeThrowsPercentage', 'reboundsDefensive', 'reboundsOffensive',
       'reboundsTotal', 'foulsPersonal', 'turnovers', 'plusMinusPoints'],
      dtype='object')

In [5]:
players_stats['name'] = players_stats['firstName'] + " " + players_stats['lastName']

In [6]:
players_stats['gameDateTimeEst'] = pd.to_datetime(
    players_stats['gameDateTimeEst'],
    format='ISO8601',
    utc=True
)

In [7]:
players_stats['gameDateTime'] = pd.to_datetime(players_stats['gameDateTimeEst']).dt.tz_localize(None)

In [8]:
players_stats['playerTeam'] = players_stats['playerteamCity'] + " " + players_stats['playerteamName']
players_stats['opponentTeam'] = players_stats['opponentteamCity'] + " " + players_stats['opponentteamName']

In [9]:
players_stats['Label'] = (players_stats['gameLabel'].fillna('') + ' ' + players_stats['gameSubLabel'].fillna('')).str.strip()

In [10]:
players_stats['Label'].unique()

array(['', 'Emirates NBA Cup Championship',
       'Emirates NBA Cup West Semifinal',
       'Emirates NBA Cup East Semifinal',
       'Emirates NBA Cup West Quarterfinal',
       'Emirates NBA Cup East Quarterfinal',
       'Emirates NBA Cup West Group B', 'Emirates NBA Cup West Group A',
       'Emirates NBA Cup West Group C', 'Emirates NBA Cup East Group B',
       'Emirates NBA Cup East Group C', 'Emirates NBA Cup East Group A',
       'NBA Mexico City Game', 'Preseason', 'Preseason NBA China Game',
       'Preseason NBA Melbourne Game', 'Preseason NBA Abu Dhabi Game',
       'NBA Finals Game 7', 'NBA Finals Game 6', 'NBA Finals Game 5',
       'NBA Finals Game 4', 'NBA Finals Game 3', 'NBA Finals Game 2',
       'NBA Finals Game 1', 'East Conf. Finals Game 6',
       'East Conf. Finals Game 5', 'West Conf. Finals Game 5',
       'East Conf. Finals Game 4', 'West Conf. Finals Game 4',
       'East Conf. Finals Game 3', 'West Conf. Finals Game 3',
       'East Conf. Finals Game 2', 

In [ ]:
def classify_game_type(label):
    if label == '':
        return 'Regular'

    label = label.lower()

    # Play-In
    if 'play-in' in label:
        return 'PlayIn'

    # Playoffs
    playoff_keywords = [
        'first round',
        'semifinal',
        'semifinals',
        'conf. finals',
        'nba finals',
        'finals game'
    ]
    if any(k in label for k in playoff_keywords):
        return 'Playoffs'
    return None


players_stats['game_type_clean'] = players_stats['Label'].apply(classify_game_type)

In [12]:
players_stats['game_type_clean'].value_counts(dropna=False)

game_type_clean
Regular     1549538
Playoffs      80884
None          13217
PlayIn          804
Name: count, dtype: int64

In [13]:
players_stats = players_stats[players_stats['game_type_clean'].isin(['Regular', 'Playoffs', 'PlayIn'])]

In [14]:
players_stats['season'] = players_stats['gameDateTime'].apply(
    lambda x: x.year if x.month >= 10 else x.year-1
)

In [15]:
players_stats = players_stats[players_stats['season'] >= 2001]

In [16]:
players_stats['numMinutes'].dtype

dtype('float64')

In [17]:
def convert_minutes(x):
    if pd.isna(x):
        return 0
    if isinstance(x, str) and ':' in x:
        m, s = x.split(':')
        return int(m) + int(s)/60
    return float(x)

players_stats['numMinutes'] = players_stats['numMinutes'].apply(convert_minutes)

In [18]:
players_stats = players_stats[players_stats['numMinutes'] > 0]

In [19]:
players_stats['FG_pct'] = (
    players_stats['fieldGoalsMade'] /
    players_stats['fieldGoalsAttempted']
)

players_stats['3P_pct'] = (
    players_stats['threePointersMade'] /
    players_stats['threePointersAttempted']
)

players_stats['FT_pct'] = (
    players_stats['freeThrowsMade'] /
    players_stats['freeThrowsAttempted']
)

In [20]:
players_stats = players_stats.drop(columns=['firstName', 'lastName',
                                            'playerteamCity', 'playerteamName',
                                            'opponentteamCity', 'opponentteamName',
                                            'gameLabel', 'gameSubLabel', 'Label', 
                                            'gameDateTimeEst','gameType', 'threePointersPercentage',
                                            'fieldGoalsPercentage', 'freeThrowsPercentage'
                                ])

In [21]:
players_stats.columns

Index(['personId', 'gameId', 'seriesGameNumber', 'win', 'home', 'numMinutes',
       'points', 'assists', 'blocks', 'steals', 'fieldGoalsAttempted',
       'fieldGoalsMade', 'threePointersAttempted', 'threePointersMade',
       'freeThrowsAttempted', 'freeThrowsMade', 'reboundsDefensive',
       'reboundsOffensive', 'reboundsTotal', 'foulsPersonal', 'turnovers',
       'plusMinusPoints', 'name', 'gameDateTime', 'playerTeam', 'opponentTeam',
       'game_type_clean', 'season', 'FG_pct', '3P_pct', 'FT_pct'],
      dtype='object')

In [22]:
players_stats.to_csv(r"C:\Users\himan\Downloads\DA\Project\NBA_Analytics\Data\Cleaned\players_stats_clean.csv", index=False)

In [23]:
print(players_stats['season'].min(), players_stats['season'].max(),
players_stats['game_type_clean'].value_counts(),
players_stats['numMinutes'].min(),
players_stats[['FG_pct','3P_pct','FT_pct']].isna().mean())

2001 2025 game_type_clean
Regular     641974
Playoffs     40458
PlayIn         620
Name: count, dtype: int64 0.01 FG_pct    0.044084
3P_pct    0.363882
FT_pct    0.413248
dtype: float64
